In [1]:
import sys
sys.path.append("../../src/earthtext")

from earthtext.osm import multilabel
import rasterio
import xarray as xr
import os
from rlxutils import subplots
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from osm import osm
from osm import multilabel
from collections import OrderedDict
import pandas as pd
from progressbar import progressbar as pbar
from joblib import Parallel, delayed
from pyproj import CRS
import numpy as np
epsg4326 = CRS.from_epsg(4326)

get_region_hash = osm.get_region_hash

In [2]:
kvmerged = multilabel.OSMCodeSets.get('naip')['kvmerged']

In [3]:
imgs_folder    = "/opt/data/"
osmobjs_folder = "/opt/data/california-naip-chips/osm"
master_index   = "/opt/data/california-naip-chips/california-naip-chips-100k.parquet"
osm_codeset    = 'naip'

In [4]:
from importlib import reload
reload(multilabel)
iosdata = multilabel.ImageOSMData(imgs_folder=imgs_folder, osmobjs_folder=osmobjs_folder, master_index=master_index, osm_codeset=osm_codeset).init_index(compute_hashid=False)

In [5]:
iosdata.m.shape

(112320, 8)

## create the multilabels

In [6]:
splits = np.linspace(0, len(iosdata.m), 120).astype(int)
splits[-1]+=1
splits = [[splits[i], splits[i+1]] for i in range(len(splits)-1)]
#splits = np.random.permutation(splits)
#splits

In [23]:
from rlxutils import ElapsedTimes
et = ElapsedTimes()

In [36]:
def get_tags_chipcount(start, end):
    cc = {}
    cc_check = {}
    for chip_id in iosdata.m.index[start:end]:
        c = ImageOSMChip(iosdata, chip_id).read_osm()
        keyval_onehot,keyval_strs = c.get_onehot()
        if keyval_strs is None:
            continue
        # count with stags (computed by ImageOSMChip from osm dataframe)
        for stag in keyval_strs:
           if not stag in cc.keys():
               cc[stag]=0
           cc[stag]+=1
        #if keyval_strs is None:
        #    continue

        """
        # count with from osm dataframe
        ktags = {}
        for t in c.osm.tags.values:
            for k,v in t.items():
                stag = f"{k}={v}"
                if stag in kvmerged.keyvals_codes.keys():
                    if not stag in cc_check.keys():
                        cc_check[stag] = 0
                    cc_check[stag] += 1
        """         
    return cc        

k = get_tags_chipcount(5,15)

In [37]:
rr = Parallel(n_jobs=60, verbose=3)(delayed(get_tags_chipcount)(start, end) for start, end in splits)

[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  40 out of 119 | elapsed:  1.7min remaining:  3.3min
[Parallel(n_jobs=60)]: Done  80 out of 119 | elapsed:  2.8min remaining:  1.4min
[Parallel(n_jobs=60)]: Done 119 out of 119 | elapsed:  3.9min finished


In [40]:
k = pd.DataFrame(rr).fillna(0)
k

,highway=*,highway=track,landuse=*,landuse=farmland,landuse=industrial,highway=residential,natural=*,natural=scrub,waterway=*,waterway=stream,...,bridge=*,bridge=yes,place=city,industrial=*,industrial=oil,railway=light_rail,natural=coastline,reservoir_type=*,reservoir_type=water_storage,natural=valley
0,370,105,253,45.0,58.0,272.0,66.0,8.0,222.0,222.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,411,168,209,3.0,7.0,104.0,135.0,0.0,194.0,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,149,149,678,0.0,2.0,0.0,342.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,299,179,372,1.0,0.0,11.0,311.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,303,272,52,13.0,15.0,23.0,272.0,0.0,253.0,253.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,437,150,131,93.0,0.0,268.0,47.0,25.0,333.0,333.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115,343,199,99,30.0,2.0,119.0,19.0,3.0,383.0,383.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
116,360,170,226,42.0,5.0,180.0,100.0,0.0,143.0,143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
117,91,65,851,33.0,5.0,25.0,20.0,0.0,111.0,111.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
ks = k.sum(axis=0)
ks[ks<100]

aeroway=taxiway       1.0
surface=gravel       21.0
surface=dirt         39.0
crossing=marked      70.0
aeroway=runway       20.0
tunnel=*             11.0
tunnel=yes           11.0
bridge=*             11.0
bridge=yes           11.0
natural=coastline    43.0
natural=valley       11.0
dtype: float64

In [9]:
def getc(start, end):
    r = []
    for chip_id in iosdata.m.index[start:end]:
        c = multilabel.ImageOSMChip(iosdata, chip_id).read_osm()
        keyval_onehot,keyval_strs = c.get_onehot()

        if keyval_onehot is None:
            r.append([chip_id, None])
            continue
            
        onehot_area = keyval_onehot.loc['area'].values.copy()
        onehot_count = keyval_onehot.loc['count'].values.copy()
        onehot_length = keyval_onehot.loc['length'].values.copy()
        keyval_strs = [i for i in keyval_strs]
        r.append([chip_id, onehot_count, onehot_area, onehot_length, keyval_strs])
    return r

In [10]:
rr = Parallel(n_jobs=60, verbose=3)(delayed(getc)(start, end) for start, end in splits)

[Parallel(n_jobs=60)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=60)]: Done  40 out of 119 | elapsed:  1.7min remaining:  3.4min
[Parallel(n_jobs=60)]: Done  80 out of 119 | elapsed:  2.8min remaining:  1.4min
[Parallel(n_jobs=60)]: Done 119 out of 119 | elapsed:  4.0min finished


In [11]:
r = []
invalid = []
for ri in pbar(rr):
    for rii in ri:
        if rii[1] is None:
            invalid.append(rii[0])
        else:
            r.append(rii)

100% (119 of 119) |######################| Elapsed Time: 0:00:00 Time:  0:00:00


In [12]:
len(r)

104138

In [27]:
chip_id = np.random.choice(invalid)
c = multilabel.ImageOSMChip(iosdata, chip_id).read_osm()
c.osm

,tags,geometry,kind,length,area,stags,chip_id
9697,{'highway': 'service'},"LINESTRING (-118.37695 35.07491, -118.37708 35...",way,149.825149,0.0,highway: service,0b34064c291db
10155,{'highway': 'service'},"LINESTRING (-118.37741 35.07490, -118.37759 35...",way,48.574941,0.0,highway: service,0b34064c291db


In [33]:
c = iosdata.sample_chip().read_osm()
c.osm

,tags,geometry,kind,length,area,stags,chip_id
159,"{'highway': 'track', 'ohv:difficulty': 'hard',...","LINESTRING (-116.85707 34.95694, -116.85713 34...",way,100.406591,0.0,highway: track\n<br>\nohv:difficulty: hard\n<b...,33e856bce86da


In [34]:
keyval_onehot,keyval_strs = c.get_onehot()

In [35]:
keyval_onehot

,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,118
area,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
length,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
kvmerged = multilabel.OSMCodeSets.get('naip')['kvmerged']
len(kvmerged.inverse_codes)

119

In [39]:
rkv = {}
for ri in r:
    for kv in ri[4]:
        if not kv in rkv.keys():
            rkv[kv]=0
        rkv[kv] += 1

In [40]:
rkv

{'highway=*': 53393,
 'highway=track': 18249,
 'landuse=*': 37051,
 'landuse=farmland': 1766,
 'landuse=industrial': 2630,
 'highway=residential': 29933,
 'natural=*': 18187,
 'natural=scrub': 4189,
 'waterway=*': 22788,
 'waterway=stream': 22071,
 'landuse=orchard': 877,
 'natural=water': 3075,
 'water=*': 2420,
 'water=canal': 114,
 'landuse=meadow': 1980,
 'water=basin': 373,
 'amenity=*': 4590,
 'amenity=parking': 2776,
 'highway=footway': 5374,
 'landuse=retail': 1576,
 'parking=*': 1276,
 'parking=surface': 1276,
 'building=*': 22959,
 'building=commercial': 2339,
 'surface=*': 1293,
 'surface=asphalt': 710,
 'building=yes': 14047,
 'building=warehouse': 1312,
 'building=industrial': 1728,
 'landuse=commercial': 1561,
 'water=reservoir': 563,
 'amenity=school': 1925,
 'power=*': 968,
 'power=generator': 331,
 'power=plant': 547,
 'landuse=farmyard': 373,
 'landuse=farm': 319,
 'landuse=residential': 13141,
 'aeroway=*': 767,
 'aeroway=aerodrome': 764,
 'natural=wetland': 158,
 'l

## create dataframe with multilabels

In [41]:
len(r)

104138

In [42]:
rr = pd.DataFrame(r, columns=['chip_id', 'onehot_count', 'onehot_area', 'onehot_length', 'string_labels'])
rr.set_index('chip_id', inplace=True)
rr['onehot_count'] = [i.astype(np.int32) for i in rr.onehot_count.values]

In [43]:
rr.head()

,onehot_count,onehot_area,onehot_length,string_labels
chip_id,,,,
363a8bd1189d2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=track, landuse=*, landuse=..."
1ff73cff31463,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=farmland]"
0fa44a50e26f3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=track, landuse=*, landuse=..."
10164c9de3dbc,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=residential, natural=*, na..."
107eb5b33ddeb,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=industrial]"


In [48]:
(np.stack(rr.onehot_count.values)>0).sum(axis=0)

array([  767,   764,   123,  4590,   136,   153,  2776,  1925,   127,
       22959,  3278,   129,  2339,   309,   216, 11799,  1728,  3689,
        2029,   735,  1312, 14047,   183,   183,   305,   305, 53393,
        1269,  5374,  6519,   107, 29933, 18249,   178,   178, 37051,
        1561,   378,   319,  1766,   373,   158,   981,  2630,   174,
        1980, 13827,   877,   483,   146,   223, 13141,  1576,  6975,
         233,   537,  2174,  2510,  1213,   333,   149,  1153, 18187,
         290,  5104,   331,  1134,   143,   165,  2323,   379,  4189,
        3075,   158,  1631,  1276,  1276,   260,   117,   143,   968,
         331,   547,   196,   103,   103,   586,   334,   260,   956,
         390,   291,   152,   327,  1293,   710,   287,   134,   163,
         109,   389,   110,   182,   103,  2420,   373,   114,   713,
         136,   563,   478,   121,   113,  1821,   117,  1730, 22788,
         776, 22071])

## join with original metadata dataframe

In [49]:
rohem = gpd.GeoDataFrame(rr.join(iosdata.m[[c for c in iosdata.m.columns if not c in ['onehot_count', 'onehot_area', 'onehot_length', 'string_labels']]]), crs=epsg4326)
rohem


,onehot_count,onehot_area,onehot_length,string_labels,date,chip_index_x,chip_index_y,cloud_cover_percentage,nodata_percentage,geometry,embeddings,original_chip_id
chip_id,,,,,,,,,,,,
363a8bd1189d2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=track, landuse=*, landuse=...",2022-06-07,39,27,0.0,0.0,"POLYGON ((-118.87306 34.96546, -118.87303 34.9...","[-0.15684646, -0.012374564, 0.14199159, 0.0223...",ca_m_3411801_ne_11_060_20220607-39-27
1ff73cff31463,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=farmland]",2022-06-07,32,24,0.0,0.0,"POLYGON ((-118.88492 34.96943, -118.88489 34.9...","[-0.1360133, 0.01871811, 0.14964038, 0.0063366...",ca_m_3411801_ne_11_060_20220607-32-24
0fa44a50e26f3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=track, landuse=*, landuse=...",2022-06-07,17,6,0.0,0.0,"POLYGON ((-118.91073 34.99395, -118.91070 34.9...","[-0.15236035, 0.020283332, 0.15849304, 0.01576...",ca_m_3411801_ne_11_060_20220607-17-6
10164c9de3dbc,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[highway=*, highway=residential, natural=*, na...",2022-06-07,24,10,0.0,0.0,"POLYGON ((-118.89883 34.98860, -118.89879 34.9...","[-0.0994814, 0.021638228, 0.059680425, -0.0212...",ca_m_3411801_ne_11_060_20220607-24-10
107eb5b33ddeb,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=industrial]",2022-06-07,25,3,0.0,0.0,"POLYGON ((-118.89737 34.99832, -118.89734 34.9...","[-0.13629381, 0.022764312, 0.14250448, 0.00573...",ca_m_3411801_ne_11_060_20220607-25-3
...,...,...,...,...,...,...,...,...,...,...,...,...
139d06c9ad49a,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=military]",2022-05-01,33,0,0.0,0.0,"POLYGON ((-117.19488 35.43942, -117.19488 35.4...","[-0.14007863, 0.027444532, 0.15230183, 0.00352...",ca_m_3511739_sw_11_060_20220501-33-0
1f1d514789561,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=military]",2022-05-01,29,3,0.0,0.0,"POLYGON ((-117.20164 35.43525, -117.20164 35.4...","[-0.14104775, 0.025011389, 0.1589857, 0.019384...",ca_m_3511739_sw_11_060_20220501-29-3
2297da4bc0278,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[landuse=*, landuse=military]",2022-05-01,16,21,0.0,0.0,"POLYGON ((-117.22357 35.41028, -117.22357 35.4...","[-0.13999236, 0.023521734, 0.1541872, -0.01373...",ca_m_3511739_sw_11_060_20220501-16-21


In [50]:
rohem.to_parquet(master_index)

In [32]:
(np.stack(rohem['onehot_count'].values)>0).sum(axis=0)

array([  788,   764,   123,    20,     1,  4590,   136,   153,  2776,
        1925,   127,    11,    11, 22959,  3278,   129,  2339,   309,
         216, 11799,  1728,  3689,  2029,   735,  1312, 14047,   250,
          70,   183,   305,   305,  2774,    11,   855,    53,    63,
         323,   121,   178,   178, 37051,  1561,   378,   319,  1766,
         373,   158,   981,  2630,   174,  1980, 13827,   877,   483,
         146,   223, 13141,  1576,  6975,   233,   537,  2174,  2510,
        1213,   333,   149,  1153, 18241,   290,    43,  5104,   331,
        1134,   143,   165,  2323,   379,  4189,    11,  3075,   158,
        1631,  1276,  1276,   260,   117,   143,   968,   331,   547,
         196,   103,   103,   586,   334,   260,   956,   390,   291,
         152,   327,  1338,   710,   287,    39,   134,    21,   163,
         109,   389,   110,   182,   103,    11,    11,  2420,   373,
         114,   713,   136,   563,   478,   121,  1420,   113,  1821,
         117,  1730,

In [51]:
(np.stack(rohem['onehot_count'].values)>0).sum(axis=0)

array([  767,   764,   123,  4590,   136,   153,  2776,  1925,   127,
       22959,  3278,   129,  2339,   309,   216, 11799,  1728,  3689,
        2029,   735,  1312, 14047,   183,   183,   305,   305, 53393,
        1269,  5374,  6519,   107, 29933, 18249,   178,   178, 37051,
        1561,   378,   319,  1766,   373,   158,   981,  2630,   174,
        1980, 13827,   877,   483,   146,   223, 13141,  1576,  6975,
         233,   537,  2174,  2510,  1213,   333,   149,  1153, 18187,
         290,  5104,   331,  1134,   143,   165,  2323,   379,  4189,
        3075,   158,  1631,  1276,  1276,   260,   117,   143,   968,
         331,   547,   196,   103,   103,   586,   334,   260,   956,
         390,   291,   152,   327,  1293,   710,   287,   134,   163,
         109,   389,   110,   182,   103,  2420,   373,   114,   713,
         136,   563,   478,   121,   113,  1821,   117,  1730, 22788,
         776, 22071])

## visualize some chip

In [24]:
chip_id = rohem.index[np.random.randint(len(rohem))]
c = multilabel.ImageOSMChip(iosdata, chip_id).read_osm()

In [25]:
c.osm

,tags,geometry,kind,length,area,stags,chip_id
478,{'natural': 'sand'},"POLYGON ((-117.37409 34.98933, -117.37408 34.9...",way,614.86567,23628.737047,natural: sand,1452c1b5a9fcc


In [44]:
keyval_onehot,keyval_strs = c.get_onehot()